In [5]:
import sys
import os
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam as LegacyAdam
from keras.regularizers import l2

df = pd.read_csv(r'/Users/admin/Documents/Emotion Detection/fer2013.csv')

X_train, train_y, X_test, test_y = [], [], [], []
for index, row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            X_train.append(np.array(val, 'float32'))
            train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            X_test.append(np.array(val, 'float32'))
            test_y.append(row['emotion'])
    except Exception as e:
        print(f"Error occurred at index :{index} and row:{row}, error: {e}")

# Data preprocessing and normalization moved up here for early checks
X_train = np.array(X_train, 'float32')
train_y = np.array(train_y, 'float32')
X_test = np.array(X_test, 'float32')
test_y = np.array(test_y, 'float32')

# Convert labels to categorical
train_y = keras.utils.to_categorical(train_y, num_classes=7)
test_y = keras.utils.to_categorical(test_y, num_classes=7)

# Normalize the data
X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)
X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

# Reshape for CNN input
X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)
X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

# Model definition...
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(256, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))  # 7 classes for the emotion labels

#Model Compile......

model.compile(loss=categorical_crossentropy, optimizer=LegacyAdam(), metrics=['accuracy'])
if X_train.size == 0 or train_y.size == 0:
    raise ValueError("Training data or labels are empty.")
if X_test.size == 0 or test_y.size == 0:
    raise ValueError("Test data or labels are empty.")
model.fit(X_train, train_y, batch_size=32, epochs=30, verbose=1, validation_data=(X_test, test_y), shuffle=True)

loss_and_metrics = model.evaluate(X_test, test_y,)
print(loss_and_metrics)

# Save model...
fer_json = model.to_json()
with open("fer.json", "w") as json_file:
    json_file.write(fer_json)

# Save the model weights to H5
model.save_weights("fer.h5")

# Print confirmation message
print("Saved model to disk")

/Users/admin/miniconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/admin/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)
/Users/admin/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:206: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/admin/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:163: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/Users/admin/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:198: RuntimeWarning: invalid value encountered in divide
  ret = ret.dtype.type(ret / rcount)


ValueError: Test data or labels are empty.